<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/02%20HW_lab/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
import pandas as pd
import json
import numpy as np

# Подготовка данных для анализа

1. Скачаем реестр анализируемых компаний
2. Прочитаем Excel файл и создадим из него DataFrame. Считывать будем только колонки, которые будут использоваться в дальнейших расчетах показателя
[EBITDA](https://xn--80aapampemcchfmo7a3c9ehj.xn--p1ai/news/chto-takoe-ebitda/)


In [ ]:
!wget "https://github.com/aleks-haksly/Postypashka/raw/main/DA/02%20HW/%D0%A0%D0%B5%D0%B5%D1%81%D1%82%D1%80.xlsx" -q

In [ ]:
df_reestr = pd.read_excel("/content/Реестр.xlsx", header = 2, usecols=[1, 2, 3, 5, 6]) # nrows=1000

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


3. Отфильтруем DataFrame и сохраним его копию, содержащую данные отвечающие всем условиям:
* Юридическое лицо
* Среднее предприятие либо Малое предприятие
* Основной вид деятельности - "41.20 Строительство жилых и нежилых зданий"

In [ ]:
df_reestr = df_reestr.query("(`Категория` in ('Среднее предприятие', 'Малое предприятие')) & \
                             (`Основной вид деятельности` == '41.20 Строительство жилых и нежилых зданий') &\
                             (`Тип субъекта` == 'Юридическое лицо')")[['Наименование / ФИО', 'ИНН']]

In [ ]:
df_reestr.to_excel("reestr_filtered.xlsx")

In [ ]:
!wget "https://github.com/aleks-haksly/Postypashka/raw/main/DA/02%20HW_lab/reestr_filtered.xlsx" -q

In [ ]:
df_reestr = pd.read_excel("reestr_filtered.xlsx", index_col=[0])

# Парсинг данных

Парсинг будет состоять из 2 этапов:
1. Для кажого предприятия, находящегося в нашем DataFrame мы по ИНН получим его внутренний `id` сайта nalog.ru
2. Используя найденные `id` мы выгрузим и аггрегируем ифнормацию об изменени **EBITDA** компаний во времени

## Этап 1 - получения внтренних id компаний

In [ ]:
df_reestr.head()

,Наименование / ФИО,ИНН
19,"""КОРПОРАЦИЯ ВИТ"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕ...",5038038838
51,"""ХОЗРАСЧЕТНАЯ СТРОИТЕЛЬНО-ТЕХНОЛОГИЧЕСКАЯ ФИРМ...",5027006369
4652,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""777""",1414006922
4667,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОТЕХИМПОРТ""",3327332190
4673,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АКС""",7816061297


In [2]:
headers ={'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
session = requests.Session()
session.headers = headers

In [3]:
res = session.get(f"https://bo.nalog.ru/advanced-search/organizations/search?query=5038038838&page=0").json()
org_id = res["content"][0]["id"]


ConnectTimeout: HTTPSConnectionPool(host='bo.nalog.ru', port=443): Max retries exceeded with url: /advanced-search/organizations/search?query=5038038838&page=0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d847878f790>, 'Connection to bo.nalog.ru timed out. (connect timeout=None)'))

In [5]:
 df_ebitda = pd.DataFrame.from_records(json.load(open('EBITDA.json', 'r'))[1:])
 df_ebitda.head()

,id,2023,2022,2021,2019,2020
0,8617989,52209.0,35173.0,41048.0,11181.0,25518.0
1,6225927,NaN,NaN,NaN,NaN,NaN
2,3964947,16336.0,22898.0,57133.0,8189.0,29107.0
3,10780694,20405.0,35578.0,42603.0,NaN,8185.0
4,8454170,127222.0,27800.0,6244.0,NaN,NaN


In [13]:
df_reestr = pd.read_excel("reestr_filtered.xlsx")
df_reestr.head()

,Наименование / ФИО,ИНН,Категория
0,"""КОРПОРАЦИЯ ВИТ"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕ...",5038038838,Малое предприятие
1,"""ХОЗРАСЧЕТНАЯ СТРОИТЕЛЬНО-ТЕХНОЛОГИЧЕСКАЯ ФИРМ...",5027006369,Среднее предприятие
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""777""",1414006922,Малое предприятие
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОТЕХИМПОРТ""",3327332190,Малое предприятие
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АКС""",7816061297,Малое предприятие


In [25]:
ids = json.load(open('company_ids.json', 'r'))
df_ids = pd.DataFrame(ids, columns=['ИНН', 'id']).dropna().astype({"ИНН": "Int64", "id": "Int64"})
df_ids.head()

,ИНН,id
0,5038038838,5872486
1,5027006369,6900709
2,1414006922,4980670
3,3327332190,3867230
4,7816061297,7041303


In [44]:
data = df_reestr.merge(df_ids, how='left', on='ИНН')\
                .merge(df_ebitda, how='left', on='id')\
                .iloc[:,[0,2,1, 3, 7, 8, 6, 5, 4]]\
                .dropna( subset=['2023', '2022', '2021'])


In [45]:
data

,Наименование / ФИО,Категория,ИНН,id,2019,2020,2021,2022,2023
0,"""КОРПОРАЦИЯ ВИТ"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕ...",Малое предприятие,5038038838,5872486,559815.0,9571594.0,781779.0,481207.0,451838.0
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОТЕХИМПОРТ""",Малое предприятие,3327332190,3867230,NaN,49143.0,64139.0,88505.0,146457.0
12,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АРХАНГЕЛЬСКГРАЖДАНРЕКОНС...",Малое предприятие,2901030646,1410906,NaN,129977.0,73080.0,-78613.0,-18412.0
13,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БАВАРСКИЙ ДОМ""",Среднее предприятие,7709008852,6462814,298235.0,722341.0,825226.0,578566.0,340754.0
14,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БАЙКАЛСТРОЙГРАД""",Среднее предприятие,3811167535,7906828,NaN,25392.0,34943.0,4498.0,8123.0
...,...,...,...,...,...,...,...,...,...
8916,"ПРОИЗВОДСТВЕННЫЙ КООПЕРАТИВ ""ЭВЕРЕСТ""",Малое предприятие,2411028648,10653011,NaN,NaN,112979.0,145637.0,90214.0
8919,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""САХАЛИНМОРНЕФТ...",Малое предприятие,6506011671,10105628,223031.0,190570.0,123756.0,114799.0,155964.0
8920,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СПЕЦИАЛИЗИРОВА...",Среднее предприятие,5751005940,2824350,NaN,1233757.0,1533092.0,1209321.0,1018935.0
8921,"РЕМОНТНО-СТРОИТЕЛЬНАЯ ФИРМА ""МИР"" ОБЩЕСТВО С О...",Малое предприятие,2010001470,397366,NaN,NaN,5521.0,8200.0,11248.0
